### Fetching Data

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import pathlib
import numpy as np
import os
import json
import cv2
from matplotlib import pyplot as plt
import zipfile
import torch
import torchvision
from torchvision import datasets, transforms
import torchvision
from torchvision import datasets, transforms
from sp_module import Net
from train import Network_Trainer
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.optim as optim
import torch.nn as nn
from DataSet import JerseyDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision.transforms import v2
from torchvision.transforms import InterpolationMode
from torch.nn import functional as F

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [56]:
# from SoccerNet.Downloader import SoccerNetDownloader as SNdl
# mySNdl = SNdl(LocalDirectory="SoccerNet")
# mySNdl.downloadDataTask(task="jersey-2023", split=["train","test","challenge"])

In [57]:
"""
This function will load the data and return two zipped arrays: 

output:       contains an image stored as a numpy array
class_labels: contains the corresponding label for each element of output

@params
dir: should be the path that contains both train images, and json dic with labels
save: If you want it to save the output as a zip file (so you don't have to do this twice)
"""

def load_data(dir, save = False):
    #labels is a dictionary mapping file_numer ->class_label
    with open(dir + "/train_gt.json") as file:
        labels = json.load(file)

    #converts an image to a numpy array
        
    """
    Eventually we will want to resize the images when we localize. This is just so that they are all the same size.
    """
    get_image = lambda file_name: cv2.resize(cv2.imread(file_name),(40, 100))

    output = []
    class_labels = []

    #iterate through the folders, convert the images to RGB arrays, and then append the class label
    for folder in list(os.listdir(dir+"/images")):

        if folder == '.DS_Store': continue
        #if (folder != '1' and folder != '2'): continue

        cls = labels[folder]
        images = os.listdir(os.path.join(dir+"/images", folder))

        for image in images:
            output.append(get_image(os.path.join(dir+"/images", folder, image)))
            class_labels.append(cls)

    zip_file = zip(output, class_labels)
    if save:
        np.savez_compressed(dir+"/numpy_data.npz", output, labels)
        
    return zip_file


def load_data_from_list(dir_train, dir_test, train_list = None, test_list = None):
        

    train_images = []
    train_targets=[]

    test_images = []
    test_targets = []

    get_image = lambda file_name: cv2.resize(cv2.imread(file_name),(40, 100))

    #get train labels
    with open('/Users/omarafifi/Downloads/Jersey Detection/Notebooks And Modules/SoccerNet/jersey-2023/train/train_gt.json') as train_file:
        train_labels = json.load(train_file)

    #get test labels
    with open('/Users/omarafifi/Downloads/Jersey Detection/Notebooks And Modules/SoccerNet/jersey-2023/test/test_gt.json') as test_file:
        test_labels = json.load(test_file)
    
    
    # cls = labels[folder]


    #get all the training images
    for index, train_image_name in enumerate(train_list):


        file_number = train_image_name.split('_')[0]

        image_path = dir_train + '/images/'+ file_number + '/'+ train_image_name

        class_label = train_labels[file_number]
        train_targets.append(class_label)

        train_images.append(get_image(image_path))

    for index, test_image_name in enumerate(test_list):

        file_number = test_image_name.split('_')[0]

        image_path = dir_test + '/images/'+ file_number + '/'+ test_image_name

        class_label = test_labels[file_number]
        test_targets.append(class_label)

        test_images.append(get_image(image_path))

    return train_images, train_targets, test_images, test_targets

#load the google data
def load_google_data(dir):

    dir = '/Users/omarafifi/Downloads/Jersey Detection/Data/Google Images'

    with open(dir + "/google_gt.json") as file:
        google_labels = json.load(file)

    X = []
    y = []

    dir = '/Users/omarafifi/Downloads/Jersey Detection/Data/Google Images/google_jerseys'

    for folder in list(os.listdir(dir)):
            
            get_image = lambda file_name: cv2.resize(cv2.imread(file_name),(32,32))

            if folder == '.DS_Store': continue


            #if (folder != '1' and folder != '2'): continue
            images = os.listdir(os.path.join(dir, folder))
            #print(images)

            for image in images:

                if image == None: continue
                if image == '.DS_Store': continue

                X.append(get_image(os.path.join(dir, folder, image)))
                y.append(google_labels[folder+'_'+image])

    return X, y

#introduce some of the google images into the training and test data to ensure the same number of labels
def stratify_google(X_trn, X_tst, X_goog, y_trn, y_tst, y_goog ):

    X_goog_tr, X_goog_tst, y_goog_tr, y_goog_tst = train_test_split(X_goog, y_goog, stratify= y_goog, train_size=.9)
    X_train = np.concatenate((X_trn, X_goog_tr), axis = 0)
    X_test = np.concatenate((X_tst, X_goog_tst), axis = 0)
    y_train = np.concatenate((y_trn, y_goog_tr), axis = 0)
    y_test = np.concatenate((y_tst, y_goog_tst), axis = 0)

    return X_train, X_test, y_train, y_test
def load_small_data(dir):

    data = np.load(dir)
    lst = data.files
    X = data[lst[0]]
    y = data[lst[1]]

    return X,y

def resize(width, height, images):
    output = []

    for image in images: 
        output.append(cv2.resize(image, (width, height)))
    return output


def crop_images(data):

    Images = data.copy()
    
    def crop_bottom(image, percent):
        height, width = image.shape[:2]
        cutoff = int(height * (1 - percent))
        cropped_image = image[:cutoff, :]
        return cropped_image

    def crop_top(image, percent):
        height, width = image.shape[:2]
        cutoff = int(height * percent)
        cropped_image = image[cutoff:, :]
        return cropped_image

    def crop_lr(image, percent):
        height, width = image.shape[:2]
        crop_width = int(width * percent)
        cropped_image = image[:, crop_width:-crop_width]
        return cropped_image


    for index, img in enumerate(Images):
        
        img= crop_bottom(img, 0.5)
        img = crop_top(img, 0.3)
        img = crop_lr(img, 0.2)
        img = cv2.resize(img, (32,32))
        Images[index] = img


    return Images

def augment(X):
    out = []
    for img in X:
        scale = 3.0
        height, width = img.shape[:2]
        # Upscale
        scale = 3.0
        img = cv2.resize(img, (round(scale * width), round(scale * height)), interpolation=cv2.INTER_CUBIC)
        img = cv2.GaussianBlur(img, (5, 5), 0)
        out.append(img)

    return np.array(out)


### Getting the data

In [58]:
# dir_train = '/Users/omarafifi/Downloads/Jersey Detection/Data/SoccerNet/jersey-2023/train'
# dir_test='/Users/omarafifi/Downloads/Jersey Detection/Data/SoccerNet/jersey-2023/test'
# train_list = np.genfromtxt('/Users/omarafifi/Downloads/Jersey Detection/Data/Good Images/train_img_with_numbers.txt', dtype='str')
# test_list = np.genfromtxt('/Users/omarafifi/Downloads/Jersey Detection/Data/Good Images/test_img_with_numbers.txt', dtype = 'str')
dir_train = '/Users/omarafifi/Downloads/Jersey Detection/Notebooks And Modules/SoccerNet/jersey-2023/train'
dir_test='/Users/omarafifi/Downloads/Jersey Detection/Notebooks And Modules/SoccerNet/jersey-2023/test'
train_list = np.genfromtxt('/Users/omarafifi/Downloads/Jersey Detection/Data/Paddle Output/paddle_train.csv', dtype='str')
test_list = np.genfromtxt('/Users/omarafifi/Downloads/Jersey Detection/Data/Paddle Output/paddle_test.csv', dtype = 'str')

X_trn, y_train, X_tst, y_test = load_data_from_list(dir_train=dir_train,
                    dir_test= dir_test, train_list=train_list, test_list=test_list)

with open("/Users/omarafifi/Downloads/Jersey Detection/Notebooks And Modules/SoccerNet/jersey-2023/train/train_gt.json") as file:
    train_labels = json.load(file)
with open("/Users/omarafifi/Downloads/Jersey Detection/Notebooks And Modules/SoccerNet/jersey-2023/test/test_gt.json") as file:
    test_labels = json.load(file)

X_google, y_google = load_google_data('/Users/omarafifi/Downloads/Jersey Detection/Data/Google Images/google_jerseys')
X_train, X_test = crop_images(X_trn), crop_images(X_tst)
X_train, X_test, y_train, y_test = stratify_google(X_train, X_test, X_google, y_train, y_test, y_google )
X_train, X_test = augment(X_train), augment(X_test)

#Recover the label mapping
label_to_int = {}
int_to_label = {}

for index, label in enumerate(np.unique(y_train)):
    label_to_int[label]=index
    int_to_label[index]=label

### Preprocessing

In [ ]:
# right now, there is no data processing. 
# But, We need to make sure that all the images have the same shape.
# and one hot encode y labels
def process_data(X_data, y_data):

    """
    Prepare the response:

    This part of the function just basically maps the classes to the set {0, ... 44}, and then one-hot encodes the response
    """
    #for to_categorical to work, we need to map the labels to {0, ... 57}
    label_to_int = {}
    int_to_label = {}

    for index, label in enumerate(np.unique(y_data)):
        label_to_int[label]=index
        int_to_label[index]=label

    
    y_data_new = np.vectorize(label_to_int.__getitem__)(y_data)
    y_data_new = to_categorical(y_data_new)

    #X_train, y_train = zip(*load_data(directory, True))
    X_data = np.array(X_data).reshape(-1, 3, 96, 96)


    return X_data, y_data_new


In [ ]:
X_train, y_train = process_data(X_train, y_train)
X_test, y_test = process_data(X_test, y_test)

### The Spacial Transformer Network:

#### Choosing Hyperparamaters for training

In [ ]:
means = X_train.mean(axis=(0,2,3))/255
devs = X_train.std(axis=(0,2,3))/255

In [ ]:
learning_rate = 0.001
epochs = 1
batch_size =  24

#need to compute the means and standard deviations to batch normalize: 


sp_transform = transforms.Compose([
                    transforms.ToTensor(),
                    v2.RandomHorizontalFlip(p=0.5),
                    transforms.Normalize(means,devs)
                    ])

device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


#### Prepping the data for training

In [ ]:
spt = Net()
optimizer = optim.SGD(spt.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

train_tensor_x = torch.Tensor(X_train) # transform to torch tensor
train_tensor_y = torch.Tensor(y_train)

val_tensor_x = torch.Tensor(X_test) # transform to torch tensor
val_tensor_y = torch.Tensor(y_test)

train_dataset = TensorDataset(train_tensor_x, train_tensor_y)
val_dataset = TensorDataset(val_tensor_x, val_tensor_y)

trainer = Network_Trainer(model = spt, 
                          transforms = sp_transform, criterion = criterion, 
                          epochs = epochs, device = device, optimizer = optimizer,
                          batch_size = batch_size, 
                          learning_rate = learning_rate, out_path = '/Users/omarafifi/Downloads/Jersey Detection/Data/Output Images')

In [ ]:
train_data = JerseyDataset(X_train, y_train[:100], sp_transform)
val_data = JerseyDataset(X_test[:100], y_test[:100], sp_transform)

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True )
val_loader = DataLoader(val_data, batch_size = batch_size, shuffle = False)

In [ ]:
trainer.train_model(train_loader, train_data, val_loader, val_data)

Epoch 1 of 1
Training


5it [00:01,  4.72it/s]                       


Validating


5it [00:00, 18.70it/s]                       


Train Loss: 0.2069, Train Acc: 0.00
Validation Loss: 0.2018, Val Acc: 0.00


### Load Challenge Data and Obtain The Predictions

In [ ]:
challenge_list = np.genfromtxt('/Users/omarafifi/Downloads/Jersey Detection/Data/Paddle Output/paddle_challenge.csv', dtype = 'str')
pth = '/Users/omarafifi/Downloads/Jersey Detection/Notebooks And Modules/SoccerNet/jersey-2023/challenge/images'

def get_challenge_data(challenge_list, path):

    get_image = lambda file_name: cv2.resize(cv2.imread(file_name),(40, 100))
    folders = []

    X_challenge =[]

    print('Getting Folder Names')
    for i in range(1426):
        i = str(i)
        out = np.flatnonzero(np.char.startswith(challenge_list, i + '_'))
        folders.append(challenge_list[out])



    print('Getting Images')
    for index, folder in enumerate(folders):
        #contains a list of images
        point = []
        for image in folder:

            image_path = path + '/' + str(index) + '/' + image
            point.append(get_image(image_path))

        point = crop_images(point)
        point = augment(point)

        X_challenge.append(point.reshape(-1, 3, 96, 96))

    return  X_challenge


def get_predictions(X, trainer):
    predictions = []
    print('Getting Predictions')
    for image_sequence in X:
        softmax, preds = trainer.predict(torch.tensor(image_sequence.astype(np.float32)))
        most_common = np.bincount(preds).argmax()
        predictions.append(most_common)

    return predictions




In [ ]:
X_challenge = get_challenge_data(challenge_list, pth)
y = get_predictions(X_challenge, trainer)
challenge_predictions = np.vectorize(int_to_label.__getitem__)(y)

Getting Folder Names
Getting Images
Getting Predictions


In [ ]:
challenge_predictions

array(['78', '78', '78', ..., '78', '78', '78'], dtype='<U2')